# Построение рекомендательной системы

В этом нотбуке мы выполним наш проект — построим рекомендательную систему, основанную на методах, которые мы обсудили на занятии.

Для этого мы будем использовать данные от Амазон. Граф размещен в отдельном файле, он загружается в первом блоке.

Нам нужно будет реализовать три метода, обсуждавшиеся на занятии, и протестировать их. Но общий подход во всех трех методах один и тот же: 
1. мы фиксируем вершину (в коде ниже это переменная `query`); 
2. удаляем некоторые смежные с ней ребра (в коде ниже это список `samp`); 
3. вычисляем специально определенное расстояние между нашей вершиной и всеми остальными (методы различаются как раз выбором расстояния);
4. выбираем вершины с наименьшим расстоянием до выбранной, это те вершины, в которые метод предлагает провести ребра;
5. сравниваем предложенные методом ребра с удаленными, чем больше совпадений, тем лучше сработал метод.

Вспомогательные шаги уже реализованы ниже. Шаг 2 реализован в функции `generate_graph`. Шаги 4-5 реализованы в функции `check_answer`. Вам нужно реализовать только шаги 3 для всех методов.

В первом методе, который нужно реализовать, расстоянием является просто число общих соседей. Во втором методе нужно будет посчитать усеченные моменты достижения из выбранной вершины. Их мы подсчитываем приближенно, запуская случайное блуждание несколько раз. Сначала нужно реализовать функцию для одного случайного блуждания, затем функцию для приближенного вычисления усеченных моментов достижения. Длину блуждания мы фиксируем равной 10. В третьем методе нужно посчитать усеченные моменты достижения в вершину. Для них у нас есть рекуррентная формула. Наконец, в последнем методе нужно просто посчитать суммы результатов двух предыдущих методов.

Мы подробно обсуждали все эти методы на занятии. Ниже вы также можете найти поясняющие комментарии.

Добавьте ваше решение между строками "BEGIN SOLUTION" и "END SOLUTION". Желательно не менять остальной код.

---
**Правила сдачи и оценивания.** Это задание является проектом курса, оно оценивается в 30 баллов.

Дедлайн по выполнению проекта --- **14 октября в 19:00**. Решения нужно отправить по адресу pygraphs.sber@gmail.com. Решения будут проверены до 19:00 15 октября. 

Также можно отправить решения до **19:00 12 октября**. Тогда они будут проверены до 19:00 13 октября и в случае наличия ошибок можно будет успеть их исправить до основного дедлайна.

В задании нужно реализовать 4 метода, описанных выше, каждый из них можно реализовывать независимо от остальных (хотя последний метод использует два предыдущих, для его реализации можно использовать лишь функции, реализация которых требуется в предыдущих методах). Первый метод оценивается в 9 баллов. Второй и третий метод оцениваются в 10 баллов каждый. Четвертый метод оценивается в 1 балл.

---

In [ ]:
# В этом блоке мы загружаем граф из файла и приводим его в вид, удобный для работы

import networkx as nx

amazon = nx.read_edgelist("amazon0302.txt", create_using=nx.Graph(), nodetype=int, data=False)
amazon = nx.convert_node_labels_to_integers(amazon, ordering='decreasing degree')
nodes = amazon.number_of_nodes()

In [ ]:
# В этом блоке собраны вспомогательные функции, которые потребуются вам для выполнения задания

# Эта функция получив на вход словарь упорядочивает его по значению меток
def index_sorted(a, reverse=False):
    return sorted(range(len(a)), key=lambda k: a[k], reverse=reverse)

# Эта функция позволяет выбрать ответ из посчитанных расстояний и сравнить его с целевым значением. 
# Она выбирает нужное количество вершин с минимальным расстоянием и находит число совпадений с удаленными ребрами.
# Здесь stat — это словарь с расстояниями, а samp — количество выбираемых вершин с минимальным расстоянием.
def check_answer(stat, samp, reverse=False): 
    index_dist = index_sorted(stat, reverse)
    guess = index_dist[:len(samp)]
    return len(set(samp) & set(guess))

# Эта функция генерирует тестовый пример, удаляя данные ей ребра из графа.
# Здесь samp — количество удаляемых ребер.
def generate_graph(query, samp):
    graph = amazon.copy()
    for i in samp:
        graph.remove_edge(query, i)
    return graph

In [ ]:
# В этом блоке требуется реализовать метод числа общих соседей. 
# Функция в ячейке i списка common_neigh должна сохранить число общих соседей query и i. 
# Но есть одна тонкость: ячейку с номером query и с номерами ее соседей правильно обнулить, 
# а то нам будут рекомендовать соединить query с query или ее соседями

def common_neighbours(graph, query):
    common_neigh = [0] * nodes
    ### BEGIN SOLUTION
    for i in range(nodes):
        common_neigh[i] = 0 if graph.has_edge(query, i) else len(list(nx.common_neighbors(graph, query, i)))
    common_neigh[query] = 0
    ### END SOLUTION
    return common_neigh

In [ ]:
# На примерах в этом блоке вы можете протестировать ваше решение.
# Важно: тесты нужны для самопроверки, оцениваться будет само решение

query = 422
samp = [35561, 98891, 157171, 3060, 198304, 28054, 226896, 20673, 110999, 125875, 125877, 20342, 208996, 205186, 829, 189415, 212872, 164896, 104718, 78418]
graph = generate_graph(query, samp)

ans = common_neighbours(graph, query)
assert index_sorted(ans, reverse=True)[:len(samp)] == [829, 3060, 20673, 13141, 21150, 35561, 36377, 103988, 110999, 172699, 4863, 8961, 10572, 16003, 20342, 28054, 53201, 70084, 70323, 104718]
assert check_answer(ans, samp, reverse=True) == 8

test_query = 377
test_samp = [202525, 196341, 169969, 29141, 159961, 38249, 101144, 1157, 40361, 99572, 64355, 127194, 109845, 217286, 125972, 77367, 6658, 26295, 47705, 200935]
test_graph = generate_graph(test_query, test_samp)
assert index_sorted(common_neighbours(test_graph, test_query), reverse=True)[:len(test_samp)] == [6658, 26295, 99789, 125972, 134665, 134666, 185446, 17364, 29519, 40361, 64355, 162514, 169969, 183713, 216721, 217286, 222821, 7693, 10838, 16638]

In [ ]:
# В этом блоке требуется реализовать метод случайных блужданий.
# Обратите внимание на массив used: его можно использовать для того, чтобы проверять, посещалась ли вершина в блуждании ранее
# Причем удобно не ставить там метку того, была ли посещена вершина в текущем блуждании.
# Вместо этого можно хранить номер последней итерации, на которой была посещена вершина, и сравниваем его с текущим.

import random

def hit_distance(adjlist, query, time=10):
    # инициализация статистик
    hit_dist = [0] * nodes  # искомые расстояния
    hit_times = [0] * nodes  # количество раз, когда вершина была достигнута в блуждании
    used = [0] * nodes  # последняя итерация, на которой вершина была достигнута в блуждании
    samples = nodes // time  # количествово блужданий

    ### BEGIN SOLUTION
    for i in range(1, samples + 1):
        current_node = query
        for step in range(1, time + 1):
            current_node = random.choice(adjlist[current_node])
            if used[current_node] != i:
                hit_dist[current_node] += step
                hit_times[current_node] += 1
                used[current_node] = i
    for i in range(nodes):
        hit_dist[i] += time * (samples - hit_times[i])
        hit_dist[i] /= samples
    for elem in adjlist[query]:
        hit_dist[elem] = time + 1
    hit_dist[query] = time + 1
    ### END SOLUTION

    return hit_dist

In [ ]:
# Проверьте ваше решение

adjlist = nx.convert.to_dict_of_lists(graph)
hd = hit_distance(adjlist, query)
assert check_answer(hd, samp) >= 8

test_adjlist = nx.convert.to_dict_of_lists(test_graph)
test_hd = hit_distance(test_adjlist, test_query)
assert check_answer(test_hd, test_samp) >= 9

In [ ]:
# В этом блоке необходимо реализовать подсчет усеченных моментов достижения в вершину.
# Допишите рекуррентную функцию и постобработку (какие вершины точно не должны попасть в ответ?)
# В нашем тестовом графе нет петель, но если вы захотите потестировать свое решение на других графах,
# обратите внимание, что петля (ребро, идущее из вершины в саму себя) повышает степень вершины на 2

def truncated_hitting_time(graph, query, time=10):
    tht = [[0 for _ in range(nodes)] for _ in range(time + 1)]
    for t in range(1, time + 1):
        for vert in range(nodes):
            if vert == query:
                continue
            
            if graph.degree[vert] != 0:
                ### BEGIN SOLUTION
                for elem in graph.neighbors(vert):
                    tht[t][vert] += tht[t - 1][elem]
                tht[t][vert] /= graph.degree[vert] - 1 if graph.has_edge(vert, vert) else graph.degree[vert]
                ### END SOLUTION
            tht[t][vert] += 1

    ### BEGIN SOLUTION
    tht[time][query] = time + 1
    for elem in graph.neighbors(query):
        tht[time][elem] = time + 1
    ### END SOLUTION
    return tht[time]

In [ ]:
# Проверьте ваше решение

tht = truncated_hitting_time(graph, query)
assert index_sorted(tht)[:len(samp)] == [164896, 254021, 110999, 212872, 20673, 172699, 3060, 104718, 205186, 194186, 35561, 36377, 829, 103988, 157171, 198304, 113283, 21150, 244935, 186662]
assert check_answer(tht, samp) == 11

test_tht = truncated_hitting_time(test_graph, test_query)
assert index_sorted(test_tht)[:len(test_samp)] == [185446, 134665, 134666, 216721, 222821, 125972, 6658, 169969, 26295, 162514, 99789, 202525, 40361, 217286, 183713, 160748, 163128, 64355, 196341, 47705]

In [ ]:
# В этом блоке требуется реализовать функцию, которая принимает две разные статистики и выдает новую,
# являющуюся суммой переданных

def sum_of_stats(first, second):
    ### BEGIN SOLUTION
    return [ left + right for left, right in zip(first, second) ]
    ### END SOLUTION

In [ ]:
# Проверьте ваше решение

assert check_answer(sum_of_stats(hd, tht), samp) >= 9
assert check_answer(sum_of_stats(test_hd, test_tht), test_samp) >= 9